In [1]:
import json
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [2]:
from arch import arch_model

In [3]:
from local_models import calculate_retraining_garch_forecasts

In [4]:
with open('config.json', 'r') as file:
    config = json.load(file)

In [5]:
files_folder = config['files_folders']['processed']
research_results_folder = config['files_folders']['research_results']

# Single `fvm`s

In [6]:
file_name = 'fvm_result_2022-01-29 18-48-25.pkl'

with open(f'{research_results_folder}/{file_name}', 'rb') as file:
    fvm_res = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'research_results/fvm_result_2022-01-29 18-48-25.pkl'

In [ ]:
fvm_res['train'].shape

In [ ]:
fvm_res['test'].shape

In [ ]:
fvm_res['fvm_errors']

# Consequent params fitting

In [7]:
file_name = 'fitting_consequent_params_result_2022-02-01 19-25-09.pkl'

with open(f'{research_results_folder}/{file_name}', 'rb') as file:
    res = pd.read_pickle(file)

FileNotFoundError: [Errno 2] No such file or directory: 'research_results/fitting_consequent_params_result_2022-02-01 19-25-09.pkl'

In [ ]:
consequent_metaparams = res['fvms'][0].local_method_parameters  # res['consequent_metaparams']
p = consequent_metaparams['p']
q = consequent_metaparams['q']

train = res['train']
test = res['test']

n_train = train.shape[0]
n_test = test.shape[0]

print(n_train, n_test)

In [ ]:
train.index[0]

In [ ]:
res['antecedent_params_set']

In [ ]:
res['mses']

In [ ]:
train.plot(label='train')
test.plot(label='test')

plt.legend()
plt.grid()
plt.show()

## Errors

In [ ]:
garch_mean = 'Zero'
garch_dist = 'normal'

garch = arch_model(train,
                   mean=garch_mean,
                   vol='GARCH',
                   p=p,
                   q=q,
                   dist=garch_dist)
garch_fitted = garch.fit()

simple_garch_forecasts = garch_fitted.forecast(horizon=n_test, reindex=False).variance.iloc[0].values

mse = mean_squared_error(simple_garch_forecasts, test ** 2, squared=True)
rmse = mean_squared_error(simple_garch_forecasts, test ** 2, squared=False)
mape = mean_absolute_percentage_error(simple_garch_forecasts, test ** 2)

garch_errors = {'mse': mse, 'rmse': rmse, 'mape': mape}

In [ ]:
garch_errors

In [ ]:
res['mses']

## Plots

In [ ]:
for fvm in res['fvms']:
    plt.plot(fvm.h[:-1] ** .5, label='forecast')
    plt.plot(fvm.train_data.values, label='actual values')

    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plt.plot(fvm.h[:-1], label='forecast')
plt.plot(input_cut.values ** 2, label='actual values')

plt.legend()
plt.grid()
plt.show()

# Processing summary table

In [8]:
summary = pd.read_excel(f'{research_results_folder}/summary_table.xlsx')
summary = summary[summary.columns[1:]].copy()

## Vanilla GARCH errors

In [10]:
garch_mean = 'Zero'
garch_dist = 'normal'
garch_vol = 'GARCH'

### w/r

In [11]:
rtsi = pd.read_pickle(f'{files_folder}/rtsi.pkl')

rescale_factor = 100

ret = rtsi['log return'] * rescale_factor

In [12]:
ret['2014-01-01':].iloc[100:100 + 10]

<DATE>
2014-05-30   -1.452643
2014-06-02    2.044933
2014-06-03    0.114111
2014-06-04    0.063423
2014-06-05    0.676265
2014-06-06    1.874572
2014-06-09    0.187448
2014-06-10    0.526647
2014-06-11    0.444643
2014-06-16   -1.254435
Name: log return, dtype: float64

In [13]:
def calc_vanilla_garch_wr_mse(info, data, garch_mean, garch_vol, garch_dist):
    train_size = info.loc['Train size']
    test_size = info.loc['Test size']
    p = info.loc['p']
    q = info.loc['q']
    
    train = data[info.loc['Start date']:].iloc[:train_size]
    test = data[info.loc['Start date']:].iloc[train_size:(train_size + test_size)]
    
    forecast = calculate_retraining_garch_forecasts(train, test,
                                                    mean=garch_mean,
                                                    vol=garch_vol,
                                                    p=p,
                                                    q=q,
                                                    dist=garch_dist)
    
    mse = mean_squared_error(forecast, test ** 2, squared=True)
    
    return mse

In [14]:
summary.iloc[-6:]['Vanilla GARCH MSE (w/ retraining)'] = \
    summary.tail(6).apply(calc_vanilla_garch_wr_mse, data=ret, garch_mean=garch_mean, garch_vol=garch_vol,
                          garch_dist=garch_dist, axis='columns')

Iteration:      1,   Func. Count:      5,   Neg. LLF: 1069.9369836837966
Iteration:      2,   Func. Count:     11,   Neg. LLF: 607.5653994120503
Iteration:      3,   Func. Count:     17,   Neg. LLF: 554.9119923381014
Iteration:      4,   Func. Count:     22,   Neg. LLF: 594.7649876762451
Iteration:      5,   Func. Count:     27,   Neg. LLF: 553.1807828408238
Iteration:      6,   Func. Count:     31,   Neg. LLF: 553.1802313363015
Iteration:      7,   Func. Count:     35,   Neg. LLF: 553.1800192180779
Iteration:      8,   Func. Count:     39,   Neg. LLF: 553.1800154780788
Iteration:      9,   Func. Count:     42,   Neg. LLF: 553.1800154779289
Optimization terminated successfully    (Exit mode 0)
            Current function value: 553.1800154780788
            Iterations: 9
            Function evaluations: 42
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      5,   Neg. LLF: 1068.4892511122791
Iteration:      2,   Func. Count:     11,   Neg. LLF: 611.0175052358031

Iteration:      2,   Func. Count:     11,   Neg. LLF: 639.2450493907469
Iteration:      3,   Func. Count:     17,   Neg. LLF: 672.3125182670091
Iteration:      4,   Func. Count:     22,   Neg. LLF: 586.0975902869816
Iteration:      5,   Func. Count:     27,   Neg. LLF: 585.5293058623296
Iteration:      6,   Func. Count:     31,   Neg. LLF: 585.5275992059957
Iteration:      7,   Func. Count:     35,   Neg. LLF: 585.527497579473
Iteration:      8,   Func. Count:     39,   Neg. LLF: 585.5274536462878
Iteration:      9,   Func. Count:     42,   Neg. LLF: 585.527453646171
Optimization terminated successfully    (Exit mode 0)
            Current function value: 585.5274536462878
            Iterations: 9
            Function evaluations: 42
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      5,   Neg. LLF: 1129.4955216288515
Iteration:      2,   Func. Count:     11,   Neg. LLF: 641.574751020345
Iteration:      3,   Func. Count:     17,   Neg. LLF: 673.3761083188365
Ite

Iteration:      2,   Func. Count:     11,   Neg. LLF: 618.4109945000484
Iteration:      3,   Func. Count:     17,   Neg. LLF: 642.9427975963412
Iteration:      4,   Func. Count:     23,   Neg. LLF: 563.4810825837621
Iteration:      5,   Func. Count:     28,   Neg. LLF: 563.2709781510277
Iteration:      6,   Func. Count:     32,   Neg. LLF: 563.2705333483896
Iteration:      7,   Func. Count:     36,   Neg. LLF: 563.2704854889748
Iteration:      8,   Func. Count:     40,   Neg. LLF: 563.2704819579824
Iteration:      9,   Func. Count:     43,   Neg. LLF: 563.2704819579101
Optimization terminated successfully    (Exit mode 0)
            Current function value: 563.2704819579824
            Iterations: 9
            Function evaluations: 43
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      5,   Neg. LLF: 1082.4221762770517
Iteration:      2,   Func. Count:     11,   Neg. LLF: 620.861208296571
Iteration:      3,   Func. Count:     17,   Neg. LLF: 642.359427861317
It

Iteration:      1,   Func. Count:      5,   Neg. LLF: 1163.8190384660938
Iteration:      2,   Func. Count:     11,   Neg. LLF: 652.7480039812623
Iteration:      3,   Func. Count:     17,   Neg. LLF: 727.7435311093247
Iteration:      4,   Func. Count:     22,   Neg. LLF: 646.4096076892856
Iteration:      5,   Func. Count:     27,   Neg. LLF: 600.8846892998469
Iteration:      6,   Func. Count:     32,   Neg. LLF: 600.7956205907935
Iteration:      7,   Func. Count:     36,   Neg. LLF: 600.7946301855727
Iteration:      8,   Func. Count:     40,   Neg. LLF: 600.7946239599495
Iteration:      9,   Func. Count:     44,   Neg. LLF: 600.7946213398575
Iteration:     10,   Func. Count:     47,   Neg. LLF: 600.7946213398563
Optimization terminated successfully    (Exit mode 0)
            Current function value: 600.7946213398575
            Iterations: 10
            Function evaluations: 47
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      5,   Neg. LLF: 1163.72925679781

Iteration:      1,   Func. Count:      5,   Neg. LLF: 1231.3350643658782
Iteration:      2,   Func. Count:     11,   Neg. LLF: 689.6645848459943
Iteration:      3,   Func. Count:     17,   Neg. LLF: 893.4927499801079
Iteration:      4,   Func. Count:     22,   Neg. LLF: 682.5359000297298
Iteration:      5,   Func. Count:     27,   Neg. LLF: 638.4319777017532
Iteration:      6,   Func. Count:     32,   Neg. LLF: 638.1857231948298
Iteration:      7,   Func. Count:     36,   Neg. LLF: 638.1834868281378
Iteration:      8,   Func. Count:     40,   Neg. LLF: 638.1834724953223
Iteration:      9,   Func. Count:     44,   Neg. LLF: 638.1834710485887
Iteration:     10,   Func. Count:     47,   Neg. LLF: 638.1834710480549
Optimization terminated successfully    (Exit mode 0)
            Current function value: 638.1834710485887
            Iterations: 10
            Function evaluations: 47
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      5,   Neg. LLF: 1236.07745950229

...

Iteration:      1,   Func. Count:      5,   Neg. LLF: 315595852.340265
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1297.501943788724
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1141.0938178607346
Iteration:      4,   Func. Count:     23,   Neg. LLF: 1116.0969847154056
Iteration:      5,   Func. Count:     27,   Neg. LLF: 1116.082139705412
Iteration:      6,   Func. Count:     31,   Neg. LLF: 1116.0820006520653
Iteration:      7,   Func. Count:     34,   Neg. LLF: 1116.0820006515964
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1116.0820006520653
            Iterations: 7
            Function evaluations: 34
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      5,   Neg. LLF: 315622444.3077697
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1300.2393545115958
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1144.551578150471
Iteration:      4,   Func. Count:     23,   Neg. LLF: 1118.088781372

Iteration:      1,   Func. Count:      5,   Neg. LLF: 328689877.713057
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1385.351599490087
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1190.273214331005
Iteration:      4,   Func. Count:     22,   Neg. LLF: 1163.3422030779652
Iteration:      5,   Func. Count:     26,   Neg. LLF: 1163.328821747345
Iteration:      6,   Func. Count:     30,   Neg. LLF: 1163.3285363924492
Iteration:      7,   Func. Count:     34,   Neg. LLF: 1163.3285338858211
Iteration:      8,   Func. Count:     37,   Neg. LLF: 1163.328533886393
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1163.3285338858211
            Iterations: 8
            Function evaluations: 37
            Gradient evaluations: 8
Iteration:      1,   Func. Count:      5,   Neg. LLF: 329024345.50318897
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1388.0003698731048
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1193.163509493

Iteration:      1,   Func. Count:      5,   Neg. LLF: 334251774.31632334
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1439.5740793793148
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1228.496190322798
Iteration:      4,   Func. Count:     22,   Neg. LLF: 1200.0978599670566
Iteration:      5,   Func. Count:     26,   Neg. LLF: 1200.077839020893
Iteration:      6,   Func. Count:     30,   Neg. LLF: 1200.0774155971012
Iteration:      7,   Func. Count:     34,   Neg. LLF: 1200.0774125017128
Iteration:      8,   Func. Count:     37,   Neg. LLF: 1200.0774125024607
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1200.0774125017128
            Iterations: 8
            Function evaluations: 37
            Gradient evaluations: 8
Iteration:      1,   Func. Count:      5,   Neg. LLF: 334723905.4138161
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1442.3858168815418
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1230.846142

Iteration:      3,   Func. Count:     17,   Neg. LLF: 1260.4575895346848
Iteration:      4,   Func. Count:     22,   Neg. LLF: 1231.611132257632
Iteration:      5,   Func. Count:     26,   Neg. LLF: 1231.5773471581508
Iteration:      6,   Func. Count:     30,   Neg. LLF: 1231.5767248194875
Iteration:      7,   Func. Count:     34,   Neg. LLF: 1231.5767202765871
Iteration:      8,   Func. Count:     37,   Neg. LLF: 1231.576720276894
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1231.5767202765871
            Iterations: 8
            Function evaluations: 37
            Gradient evaluations: 8
Iteration:      1,   Func. Count:      5,   Neg. LLF: 339293805.20487285
Iteration:      2,   Func. Count:     11,   Neg. LLF: 1475.2311755172432
Iteration:      3,   Func. Count:     17,   Neg. LLF: 1262.4201634408014
Iteration:      4,   Func. Count:     22,   Neg. LLF: 1233.3694675146974
Iteration:      5,   Func. Count:     26,   Neg. LLF: 1233.34000

/var/folders/wp/ypx6h8352xv06355zp13dlx40000gn/T/ipykernel_8385/3465619562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary.iloc[-6:]['Vanilla GARCH MSE (w/ retraining)'] = \


In [15]:
summary['ratio n/r'] = summary['Vanilla GARCH MSE (no retraining)'] / summary['MSE']
summary['ratio w/r'] = summary['Vanilla GARCH MSE (w/ retraining)'] / summary['MSE']

In [16]:
summary.tail(6)

,Time,p,q,Start date,Train size,Test size,# clusters,Centers,Variances,MSE,Vanilla GARCH MSE (no retraining),Vanilla GARCH MSE (w/ retraining),ratio n/r,ratio w/r
2937,2022-02-19 18-23-14,1,1,2014-01-06,252,21,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",353.142984,273.578518,179.090080,0.774696,0.507132
2938,2022-02-19 18-34-52,1,1,2014-01-06,252,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",251.835593,201.336404,137.598461,0.799476,0.546382
2939,2022-02-19 18-40-03,1,1,2014-01-06,504,21,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",542.755351,382.644697,300.493400,0.705004,0.553644
2940,2022-02-19 18-47-09,1,1,2014-01-06,252,63,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",190.298376,141.735456,99.212028,0.744806,0.521350
2941,2022-02-19 19-02-20,1,1,2014-01-06,504,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",290.843395,203.557062,160.334966,0.699885,0.551276
2942,2022-02-19 19-25-19,1,1,2014-01-06,504,63,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",202.473256,143.124972,112.755036,0.706883,0.556889


In [17]:
train = ret['2014-01-01':].iloc[:100]
test = ret['2014-01-01':].iloc[100:110]
p = 1
q = 1

retraining_garch_forecasts = calculate_retraining_garch_forecasts(train, test,
                                                                  mean=garch_mean,
                                                                  vol=garch_vol,
                                                                  p=p,
                                                                  q=q,
                                                                  dist=garch_dist)

Iteration:      1,   Func. Count:      5,   Neg. LLF: 226.89681621963246
Iteration:      2,   Func. Count:     13,   Neg. LLF: 221.8910051904815
Iteration:      3,   Func. Count:     18,   Neg. LLF: 219.54358023824105
Iteration:      4,   Func. Count:     23,   Neg. LLF: 219.19666095502856
Iteration:      5,   Func. Count:     27,   Neg. LLF: 219.3104687252924
Iteration:      6,   Func. Count:     32,   Neg. LLF: 219.16996418124938
Iteration:      7,   Func. Count:     36,   Neg. LLF: 219.16931633136116
Iteration:      8,   Func. Count:     40,   Neg. LLF: 219.1692982434543
Iteration:      9,   Func. Count:     44,   Neg. LLF: 219.1692972098965
Iteration:     10,   Func. Count:     47,   Neg. LLF: 219.1692972098913
Optimization terminated successfully    (Exit mode 0)
            Current function value: 219.1692972098965
            Iterations: 10
            Function evaluations: 47
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      5,   Neg. LLF: 228.99872445

In [18]:
retraining_garch_forecasts

array([5.14845757, 5.13455246, 5.05948509, 4.98765364, 4.41118918,
       4.31977977, 4.05652541, 4.03279291, 3.98968755, 4.05620977])

In [19]:
mean_squared_error(retraining_garch_forecasts * 100, (test * 100) ** 2, squared=True)

359506645.0673107

In [20]:
test

<DATE>
2014-05-30   -1.452643
2014-06-02    2.044933
2014-06-03    0.114111
2014-06-04    0.063423
2014-06-05    0.676265
2014-06-06    1.874572
2014-06-09    0.187448
2014-06-10    0.526647
2014-06-11    0.444643
2014-06-16   -1.254435
Name: log return, dtype: float64

## 

In [21]:
# summary = summary[summary.columns[:-2]].copy()

In [22]:
_groupby_cols = ['Start date', 'Train size', 'Test size', '# clusters', 'Centers', 'Variances']

summary.set_index(_groupby_cols)

Time  \
Start date Train size Test size # clusters Centers              Variances                           
2014-01-06 100        10        4          [-10, -2, 2, 4]      [4, 4, 4, 4]                  NaN   
                                                                [2, 4, 4, 2]                  NaN   
                                                                [4, 2, 2, 4]                  NaN   
                      20        4          [-10, -2, 2, 4]      [4, 4, 4, 4]                  NaN   
                                                                [2, 4, 4, 2]                  NaN   
...                                                                                           ...   
           252        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  2022-02-19 18-34-52   
           504        21        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  2022-02-19 18-40-03   
           252        63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  2022-02-19 18-47-09   
           504        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  2022-02-19 19-02-20   
                      63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  2022-02-19 19-25-19   

                                                                              p  \
Start date Train size Test size # clusters Centers              Variances         
2014-01-06 100        10        4          [-10, -2, 2, 4]      [4, 4, 4, 4]  1   
                                                                [2, 4, 4, 2]  1   
                                                                [4, 2, 2, 4]  1   
                      20        4          [-10, -2, 2, 4]      [4, 4, 4, 4]  1   
                                                                [2, 4, 4, 2]  1   
...                                                                          ..   
           252        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           504        21        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           252        63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           504        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
                      63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   

                                                                              q  \
Start date Train size Test size # clusters Centers              Variances         
2014-01-06 100        10        4          [-10, -2, 2, 4]      [4, 4, 4, 4]  1   
                                                                [2, 4, 4, 2]  1   
                                                                [4, 2, 2, 4]  1   
                      20        4          [-10, -2, 2, 4]      [4, 4, 4, 4]  1   
                                                                [2, 4, 4, 2]  1   
...                                                                          ..   
           252        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           504        21        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           252        63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
           504        42        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   
                      63        4          [-7.5, -1.5, 1.5, 4] [7, 6, 3, 5]  1   

                                                                                     MSE  \
Start date Train size Test size # clusters Centers              Variances                  
2014-01-06 100        10        4          [-10, -2, 2, 4]      [4, 4, 4, 4]   10.240000   
                                                                [2, 4, 4, 2]   10.240000   
                                                                [4, 2, 2, 4]   10.240000   
                      20        4          [-10, -2, 2, 4]      [4, 4, 4, 4]   14.440000   
                                                                [2,

In [23]:
_[_.index.duplicated(keep=False)]

Time  \
Start date Train size Test size # clusters Centers              Variances                           
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 6]  2022-02-05 19-56-07   
                                                                [6, 2, 2, 6]  2022-02-05 19-56-07   
                                                                [8, 2, 2, 6]  2022-02-05 19-56-07   
                                                                [8, 2, 2, 6]  2022-02-09 05-14-12   
                                                                [6, 4, 4, 6]  2022-02-11 07-13-53   
                                                                [6, 2, 2, 6]  2022-02-11 07-45-42   

                                                                              p  \
Start date Train size Test size # clusters Centers              Variances         
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 6]  1   
                                                                [6, 2, 2, 6]  1   
                                                                [8, 2, 2, 6]  1   
                                                                [8, 2, 2, 6]  1   
                                                                [6, 4, 4, 6]  1   
                                                                [6, 2, 2, 6]  1   

                                                                              q  \
Start date Train size Test size # clusters Centers              Variances         
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 6]  1   
                                                                [6, 2, 2, 6]  1   
                                                                [8, 2, 2, 6]  1   
                                                                [8, 2, 2, 6]  1   
                                                                [6, 4, 4, 6]  1   
                                                                [6, 2, 2, 6]  1   

                                                                                    MSE  \
Start date Train size Test size # clusters Centers              Variances                 
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 6]   9.722751   
                                                                [6, 2, 2, 6]  10.244789   
                                                                [8, 2, 2, 6]  10.244789   
                                                                [8, 2, 2, 6]  10.244789   
                                                                [6, 4, 4, 6]   9.722751   
                                                                [6, 2, 2, 6]  10.244789   

                                                                              Vanilla GARCH MSE (no retraining)  \
Start date Train size Test size # clusters Centers              Variances                                         
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 6]                          17.889309   
                                                                [6, 2, 2, 6]                          17.889309   
                                                                [8, 2, 2, 6]                          17.889309   
                                                                [8, 2, 2, 6]                          17.889309   
                                                                [6, 4, 4, 6]                          17.889309   
                                                                [6, 2, 2, 6]                          17.889309   

                                                                              Vanilla GARCH MSE (w/ retraining)  \
Start date Train size Test size # clusters Centers              Variances                                         
2014-01-06 100        10        4          [-7.5, -1.5, 1.5, 4] [6, 4, 4, 

In [24]:
summary

,Time,p,q,Start date,Train size,Test size,# clusters,Centers,Variances,MSE,Vanilla GARCH MSE (no retraining),Vanilla GARCH MSE (w/ retraining),ratio n/r,ratio w/r
0,NaN,1,1,2014-01-06,100,10,4,"[-10, -2, 2, 4]","[4, 4, 4, 4]",10.240000,17.890000,NaN,1.747070,NaN
1,NaN,1,1,2014-01-06,100,10,4,"[-10, -2, 2, 4]","[2, 4, 4, 2]",10.240000,17.890000,NaN,1.747070,NaN
2,NaN,1,1,2014-01-06,100,10,4,"[-10, -2, 2, 4]","[4, 2, 2, 4]",10.240000,17.890000,NaN,1.747070,NaN
3,NaN,1,1,2014-01-06,100,20,4,"[-10, -2, 2, 4]","[4, 4, 4, 4]",14.440000,20.930000,NaN,1.449446,NaN
4,NaN,1,1,2014-01-06,100,20,4,"[-10, -2, 2, 4]","[2, 4, 4, 2]",14.440000,20.930000,NaN,1.449446,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2938,2022-02-19 18-34-52,1,1,2014-01-06,252,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",251.835593,201.336404,137.598461,0.799476,0.546382
2939,2022-02-19 18-40-03,1,1,2014-01-06,504,21,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",542.755351,382.644697,300.493400,0.705004,0.553644
2940,2022-02-19 18-47-09,1,1,2014-01-06,252,63,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",190.298376,141.735456,99.212028,0.744806,0.521350
2941,2022-02-19 19-02-20,1,1,2014-01-06,504,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",290.843395,203.557062,160.334966,0.699885,0.551276


In [29]:
summary[summary['Variances'] == '[7, 6, 3, 5]']

,Time,p,q,Start date,Train size,Test size,# clusters,Centers,Variances,MSE,Vanilla GARCH MSE (no retraining),Vanilla GARCH MSE (w/ retraining),ratio n/r,ratio w/r
1150,2022-02-10 00-57-56,1,1,2014-01-06,100,10,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",9.662553,17.889309,NaN,1.851406,NaN
2937,2022-02-19 18-23-14,1,1,2014-01-06,252,21,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",353.142984,273.578518,179.090080,0.774696,0.507132
2938,2022-02-19 18-34-52,1,1,2014-01-06,252,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",251.835593,201.336404,137.598461,0.799476,0.546382
2939,2022-02-19 18-40-03,1,1,2014-01-06,504,21,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",542.755351,382.644697,300.493400,0.705004,0.553644
2940,2022-02-19 18-47-09,1,1,2014-01-06,252,63,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",190.298376,141.735456,99.212028,0.744806,0.521350
2941,2022-02-19 19-02-20,1,1,2014-01-06,504,42,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",290.843395,203.557062,160.334966,0.699885,0.551276
2942,2022-02-19 19-25-19,1,1,2014-01-06,504,63,4,"[-7.5, -1.5, 1.5, 4]","[7, 6, 3, 5]",202.473256,143.124972,112.755036,0.706883,0.556889
